In [1]:
# Setup
%pip install python-dotenv --upgrade --quiet langchain langchain-groq

from dotenv import load_dotenv
load_dotenv()

import getpass
import os
from langchain_groq import ChatGroq

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API Key: ")

# Using Llama3.1-8b (Small/Fast) to demonstrate logic failures
llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0.0)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.5/500.5 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.1/158.1 kB 3.3 MB/s eta 0:00:00
Enter your Groq API Key: ··········


## 3. The Experiment: A Tricky Math Problem

Let's try a problem that requires multi-step logic.

**Problem:**
"Roger has 10 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many does he have now?"

In [2]:
question = "Roger has 10 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many does he have now?"

# 1. Standard Prompt (Direct Answer)
prompt_standard = f"Answer this question: {question}"
print("--- STANDARD (Llama3.1-8b) ---")
print(llm.invoke(prompt_standard).content)

--- STANDARD (Llama3.1-8b) ---
To find out how many tennis balls Roger has now, we need to add the initial number of tennis balls he had (10) to the number of tennis balls he bought (2 cans * 3 tennis balls per can).

2 cans * 3 tennis balls per can = 6 tennis balls

Now, let's add the initial number of tennis balls (10) to the number of tennis balls he bought (6):

10 + 6 = 16

So, Roger now has 16 tennis balls.


In [3]:
# 2. CoT Prompt (Magic Phrase)
prompt_cot = f"Answer this question. Let's think step by step. {question}"

print("--- Chain of Thought (Llama3.1-8b) ---")
print(llm.invoke(prompt_cot).content)

--- Chain of Thought (Llama3.1-8b) ---
To find out how many tennis balls Roger has now, we need to follow these steps:

1. Roger starts with 10 tennis balls.
2. He buys 2 more cans of tennis balls. Each can has 3 tennis balls, so he buys 2 x 3 = 6 tennis balls.
3. To find the total number of tennis balls Roger has now, we add the initial number of tennis balls (10) to the number of tennis balls he bought (6). 

10 (initial tennis balls) + 6 (new tennis balls) = 16

So, Roger now has 16 tennis balls.


In [4]:
# Setup
%pip install python-dotenv --upgrade --quiet langchain langchain-groq

from dotenv import load_dotenv
load_dotenv()

import getpass
import os
from langchain_groq import ChatGroq

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API Key: ")

# Using Llama3.1-8b
llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0.7) # Creativity needed

In [6]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

problem = "How can I be a successful billionaire?"

# Step 1: The Branch Generator
prompt_branch = ChatPromptTemplate.from_template(
    "Problem: {problem}. Give me one unique, creative solution. Solution {id}:"
)

branches = RunnableParallel(
    sol1=prompt_branch.partial(id="1") | llm | StrOutputParser(),
    sol2=prompt_branch.partial(id="2") | llm | StrOutputParser(),
    sol3=prompt_branch.partial(id="3") | llm | StrOutputParser(),
)

# Step 2: The Judge
prompt_judge = ChatPromptTemplate.from_template(
    """
    I have three proposed solutions for: '{problem}'

    1: {sol1}
    2: {sol2}
    3: {sol3}

    Act as a Business Woman. Pick the most sustainable one (not bribery) and explain why.
    """
)

# Chain: Input -> Branches -> Judge -> Output
tot_chain = (
    RunnableParallel(problem=RunnableLambda(lambda x: x), branches=branches)
    | (lambda x: {**x["branches"], "problem": x["problem"]})
    | prompt_judge
    | llm
    | StrOutputParser()
)

print("--- Tree of Thoughts (ToT) Result ---")
print(tot_chain.invoke(problem))

--- Tree of Thoughts (ToT) Result ---
As a business woman, I have carefully evaluated the three proposed solutions and I am excited to share that I believe the most sustainable one is **The "Future-Proofing" Billionaire Strategy: Invest in and Develop Revolutionary, Sustainable, and In-Demand Technologies**.

Here's why I think this solution stands out:

1. **Long-term impact**: By focusing on sustainable and in-demand technologies, this solution has the potential to create a lasting impact on the environment and society, which aligns with my values as a business woman.
2. **Scalability**: The solution highlights the importance of designing and developing technologies that can be easily scaled up to meet growing demand, which is crucial for long-term success and sustainability.
3. **Diversified revenue streams**: By investing in various technologies, such as renewable energy, sustainable food production, and AI for social good, this solution creates multiple revenue streams and reduces

In [7]:
# 1. The Generator (Divergence)
prompt_draft = ChatPromptTemplate.from_template(
    "Write a 1-sentence movie plot about: {topic}. Genre: {genre}."
)

drafts = RunnableParallel(
    draft_scifi=prompt_draft.partial(genre="Sci-Fi") | llm | StrOutputParser(),
    draft_romance=prompt_draft.partial(genre="Romance") | llm | StrOutputParser(),
    draft_horror=prompt_draft.partial(genre="Horror") | llm | StrOutputParser(),
)

# 2. The Aggregator (Convergence)
prompt_combine = ChatPromptTemplate.from_template(
    """
    I have three movie ideas for the topic '{topic}':
    1. Sci-Fi: {draft_scifi}
    2. Romance: {draft_romance}
    3. Horror: {draft_horror}

    Your task: Create a new Mega-Movie that combines the TECHNOLOGY of Sci-Fi, the PASSION of Romance, and the FEAR of Horror.
    Write one paragraph.
    """
)

# 3. The Chain
got_chain = (
    RunnableParallel(topic=RunnableLambda(lambda x: x), drafts=drafts)
    | (lambda x: {**x["drafts"], "topic": x["topic"]})
    | prompt_combine
    | llm
    | StrOutputParser()
)

print("--- Graph of Thoughts (GoT) Result ---")
print(got_chain.invoke("Time Travel"))

--- Graph of Thoughts (GoT) Result ---
In "Echoes of Eternity," a brilliant but reclusive physicist, Emma Taylor, discovers a way to manipulate time and traverse the fabric of space-time. As she experiments with her creation, a mysterious entity from a dystopian future begins to seep into her reality, threatening to destroy the world. Desperate to prevent this catastrophic future, Emma becomes obsessed with her invention, using it to travel back in time and find the source of the disturbance. However, her journey takes an unexpected turn when she encounters a charming and enigmatic artist, Max, who is living in the past. As Emma navigates the complexities of time travel and her growing feelings for Max, she must confront the dark forces manipulating her reality and the terrible consequences of altering the timeline, forcing her to make an impossible choice between saving the world and losing the love of her life.


## 4. Summary & Comparison Table

| Method | Structure | Best For... | Cost/Latency |
|--------|-----------|-------------|--------------|
| **Simple Prompt** | Input -> Output | Simple facts, summaries | ⭐ Low |
| **CoT (Chain)** | Input -> Steps -> Output | Math, Logic, Debugging | ⭐⭐ Med |
| **ToT (Tree)** | Input -> 3x Branches -> Select -> Output | Strategic decisions, Brainstorming | ⭐⭐⭐ High |
| **GoT (Graph)** | Input -> Branch -> Mix/Aggregate -> Output | Creative Writing, Research Synthesis | ⭐⭐⭐⭐ V. High |

**Recommendation:** Start with CoT. Only use ToT/GoT if CoT fails.